In [37]:
import pandas as pd
import numpy as np

In [2]:
recid_data = pd.read_pickle('recid_data.pickle')

In [7]:
recid_data.head()

,INMATE_DOC_NUMBER,SENTENCE_END,SENTENCE_START,TOTAL_SENTENCE_COUNT,TOTAL_DISCIPLINE_INFRACTIONS,ESCAPE_HISTORY_FLAG,PRIOR_INCARCERATIONS_FLAG,AGE_AT_RELEASE,RECITIVATED,MOST_SERIOUS_OFFENSE_CODE_ARMED ROBBERY,...,MOST_SERIOUS_OFFNSE_CURR_INCAR_VIOLATION CONT SUB PENAL INST,MOST_SERIOUS_OFFNSE_CURR_INCAR_VIOLATION DRUG LAWS,MOST_SERIOUS_OFFNSE_CURR_INCAR_VIOLATION PROTECTIVE ORDER,MOST_SERIOUS_OFFNSE_CURR_INCAR_WANTON INJ PER/PROP $200 & LS,MOST_SERIOUS_OFFNSE_CURR_INCAR_WANTON INJ PER/PROP GT $200,MOST_SERIOUS_OFFNSE_CURR_INCAR_WILL/WANT INJ REAL PROPERTY,MOST_SERIOUS_OFFNSE_CURR_INCAR_WORTHLESS CHECK,INMATE_IS_FELON/MISDEMEANANT_MISD.,WEEKS_SERVED,SENTENCE_END_YEAR
0,4,1984-07-11,1983-07-12,0,0,0,1,22,0,0,...,0,0,0,0,0,0,0,0,52,1984
1,6,1973-03-28,1973-01-30,0,0,0,1,21,1,0,...,0,0,0,0,0,0,0,1,8,1973
2,6,1975-08-18,1973-04-11,0,0,0,1,24,0,0,...,0,0,0,0,0,0,0,1,122,1975
3,8,1990-05-17,1990-04-09,0,0,0,1,26,0,0,...,0,0,0,0,0,0,0,0,5,1990
4,8,1994-01-26,1993-08-30,0,0,0,1,30,1,0,...,0,0,0,0,0,0,0,0,21,1994


In [39]:
recid_data.groupby('SENTENCE_END_YEAR').agg({'RECITIVATED':["count", 'mean']})[17:65]

RECITIVATED          
                        count      mean
SENTENCE_END_YEAR                      
1972                        5  0.000000
1973                     8377  0.298197
1974                     8623  0.295605
1975                     8372  0.307453
1976                    10954  0.295052
1977                    10747  0.272634
1978                     9166  0.272093
1979                     9054  0.291915
1980                     9828  0.333842
1981                    11230  0.351469
1982                    12572  0.344973
1983                    15958  0.329991
1984                    13362  0.339545
1985                    14131  0.351426
1986                    14604  0.364078
1987                    14437  0.380065
1988                    15262  0.399948
1989                    17628  0.426197
1990                    17854  0.429540
1991                    21005  0.428327
1992                    22238  0.391897
1993                    21475  0.354971
1994                    18562  0.331807
1995                    17795  0.294128
1996                    21949  0.304570
1997                    22259  0.327463
1998                    23362  0.332591
1999                    23133  0.341763
2000                    21834  0.343593
2001                    21880  0.350411
2002                    21433  0.354640
2003                    21172  0.347298
2004                    22257  0.348160
2005                    23943  0.349079
2006                    24933  0.342638
2007                    25706  0.335175
2008                    26381  0.319662
2009                    27405  0.298705
2010                    26475  0.272975
2011                    26717  0.245611
2012                    21704  0.251567
2013                    18950  0.258364
2014                    18466  0.247753
2015                    17927  0.234283
2016                    17562  0.164731
2017                    18858  0.070103
2018                    16354  0.004219
2019                     5598  0.000179

In [3]:
features = recid_data[recid_data.columns.difference(['INMATE_DOC_NUMBER', 'RECITIVATED', 'SENTENCE_START', 'SENTENCE_END', 'SENTENCE_END_YEAR'])]

In [4]:
proper_format = pd.concat([recid_data[['INMATE_DOC_NUMBER','SENTENCE_END']], features, recid_data['RECITIVATED']], axis=1)

In [5]:
proper_format.sort_values(by='SENTENCE_END', inplace=True)

In [23]:
proper_format.rename(columns={'INMATE_DOC_NUMBER': 'entity_id', 'SENTENCE_END':"as_of_date"}, inplace=True)

In [18]:
matrix_keys = range(35)

In [ ]:
trainings = []
testings = []

train_matrix_uuids = []
train_end_times = []
test_matrix_uuids = []
evaluation_start_times = []
evaluation_end_times = []
model_configs = []
num_features = []


# create and save matrices
for i in matrix_keys:
    test_as_of_date = pd.to_datetime('1980-01-01') + pd.DateOffset(years = i)
    test_begin_date = test_as_of_date - pd.DateOffset(years = 1)
    
    train_as_of_date = test_as_of_date - pd.DateOffset(years = 3)
    
    # Start the training set at 1973
    train_df = proper_format[(proper_format.as_of_date >= '1973-01-01') &
                             (proper_format.as_of_date < train_as_of_date)]
    
    test_df = proper_format[(proper_format.as_of_date >= test_begin_date) &
                            (proper_format.as_of_date < test_as_of_date)]
    
    train_uuid = 'train_{}'.format(i)
    test_uuid = 'test_{}'.format(i)
    
    trainings.append(train_uuid)
    testings.append(test_uuid)
    
    train_matrix_uuids.append(train_uuid)
    train_end_times.append(train_as_of_date)
    test_matrix_uuids.append(test_uuid)
    evaluation_start_times.append(test_begin_date)
    evaluation_end_times.append(test_as_of_date)
    model_configs.append("")
    num_features.append(features.shape[1])
    
    
    train_df.to_csv('../train_matrices/' + train_uuid + '.csv', index=False)
    test_df.to_csv('../test_matrices/'  + test_uuid  + '.csv', index=False)
    
    print(test_uuid)

test_0
test_1
test_2
test_3
test_4
test_5
test_6
test_7
test_8
test_9
test_10


In [ ]:
# Create raw_paired_matrices info file
paired_matrices_raw = pd.DataFrame(np.column_stack([train_matrix_uuids, train_end_times, 
                                                    test_matrix_uuids, evaluation_start_times,
                                                    evaluation_end_times, model_configs, num_features]),
                                   columns=['train_matrix', 'train_end_time', 'test_matrix',
                                            'evaluation_start_time', 'evaluation_end_time',
                                            'model_config', 'num_features'])

paired_matrices_raw.to_csv('../paired_matrices_raw.csv', index=False, header=False)

# Write trainings and testings file
with open('../trainings.txt', 'w') as f:
    for item in trainings:
        f.write("%s\n" % item)
        
with open('../testings.txt', 'w') as f:
    for item in testings:
        f.write("%s\n" % item)


In [41]:
paired_matrices_raw

,train_matrix,train_end_time,test_matrix,evaluation_start_time,evaluation_end_time,model_config,num_features
0,train_0,1977-01-01 00:00:00,test_0,1979-01-01 00:00:00,1980-01-01 00:00:00,,463
1,train_1,1978-01-01 00:00:00,test_1,1980-01-01 00:00:00,1981-01-01 00:00:00,,463
2,train_2,1979-01-01 00:00:00,test_2,1981-01-01 00:00:00,1982-01-01 00:00:00,,463
3,train_3,1980-01-01 00:00:00,test_3,1982-01-01 00:00:00,1983-01-01 00:00:00,,463
4,train_4,1981-01-01 00:00:00,test_4,1983-01-01 00:00:00,1984-01-01 00:00:00,,463
5,train_5,1982-01-01 00:00:00,test_5,1984-01-01 00:00:00,1985-01-01 00:00:00,,463
6,train_6,1983-01-01 00:00:00,test_6,1985-01-01 00:00:00,1986-01-01 00:00:00,,463
7,train_7,1984-01-01 00:00:00,test_7,1986-01-01 00:00:00,1987-01-01 00:00:00,,463
8,train_8,1985-01-01 00:00:00,test_8,1987-01-01 00:00:00,1988-01-01 00:00:00,,463
9,train_9,1986-01-01 00:00:00,test_9,1988-01-01 00:00:00,1989-01-01 00:00:00,,463
